# Email Parsing

Import the required libraries.

In [ ]:
import getpass
import imaplib
import email
from email.parser import HeaderParser
import re
from email.header import decode_header
import csv
import pandas as pd
from bs4 import BeautifulSoup

Login to the Gmail

In [ ]:
print('Email')
un = input()
print('Password')
pw = getpass.getpass()
conn = imaplib.IMAP4_SSL(port = '993',host = 'imap.gmail.com')
conn.login(un,pw)
conn.select('[Gmail]/Spam')
type, emaildata = conn.search(None, 'ALL')
emaillist=emaildata[0].split()

Get email headers

In [ ]:
parser = HeaderParser()
header_list=[]
msg_text=[]
for a in emaillist:
    type, emaildata2 = conn.fetch(a, '(RFC822)')
    msg = email.message_from_string(emaildata2[0][1].decode('utf-8','ignore'))
    for txt in msg.walk():
        if not txt.is_multipart():
            msg_text = txt.get_payload(decode=True).decode('utf-8','ignore')
    soup = BeautifulSoup(msg_text, "lxml")
    msg_text= soup.get_text(strip=True)
    h = parser.parsestr(emaildata2[0][1].decode('utf-8','ignore'))
    header = {}
    header['Subject']=decode_header(h['Subject'])[0][0]
    header['ARC-Authentication-Results']=h['ARC-Authentication-Results'].strip()
    header['Return-Path']=h['Return-Path'].strip()
    header['Received']=h['Received'].strip()
    header['Received-SPF']=h['Received-SPF'].strip()
    header['Date']=pd.to_datetime(h['Date'].strip())
    if 'Reply-To' in h:
        header['Reply-To']=h['Reply-To'].strip()
    else:
        header['Reply-To']=''    
    header['Content-Type']=h['Content-Type'].strip()
    header['From']=h['From'].strip()
    header['Sender-ip']= re.findall(r'(?:(?:25[0-5]|2[0-4]\d|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d?)',str(header))
    header['Message']= len(msg_text)
    header_list.append(header)

In [ ]:
keys = header_list[0].keys()
with open('spam.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(header_list)